In [4]:
import qutip
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import pickle
import time as time
import math, cmath
import sys
#import proj_ev_library as projev
#import max_entev library as meev

from IPython.display import display, Math, Latex

In [54]:
def generating_function_M_matrix(total_no_spins, cmt0_coeff_list, cbc = False):
    
    cm_list = cmt0_coeff_list
    m_matrix_list = []
    ith_cmt_list = []
    M = total_no_spins
    
    if not cbc:  
        for m in range(M):
            if m == 1: 
                m_matrix_list.append([-np.e**(-m) - np.e**(-m) + p ** np.e**(-0)] + [0]
                                    + [np.e**(-m+2)] + [0 for i in range(M-3)])
            if m == 2:
                m_matrix_list.append([0] + [-np.e**(-m) - np.e**(-m) + p ** np.e**(-0)] + [0]
                                    + [np.e**(-m+2)] + [0 for i in range(M-4)])
            if m > 2 or (m < total_no_spins - 2):
                m_matrix_list.append([0 for j in range(m-1)] + ["A"] + [0] + ["B"] + [0 for j in range(M-m)] )
    else:
        pass 
    
    return m_matrix_list

In [55]:
((generating_function_M_matrix(5, coeff_list_t0, False)))

[['A', 0, 'B', 0, 0, 0, 0, 0],
 [-0.23575888234288467, 0, 2.718281828459045, 0, 0],
 ['A', 0, 'B', 0, 0, 0, 0],
 [0, 0.2293294335267746, 0, 1.0, 0],
 [0, 'A', 0, 'B', 0, 0, 0],
 [0, 0, 'A', 0, 'B', 0, 0],
 [0, 0, 0, 'A', 0, 'B', 0]]

In [25]:
#import simpy no hace falta

total_no_spins = 49 ### este es el nÃºmero de spines en el lattice
coherences_t0_pert0 = 1. ### el coeficiente I_m^0(t) inicial, son las coherencias a tiempo 0 sin perturbaciÃ³n. Modificar dsps.
coeff_list_t0 = [coherences_t0_pert0 + np.random.rand() for i in range(total_no_spins)] ### una configuraciÃ³n inicial 
p = .5 ### el tÃ©rmino que pondera el avg del hamiltoniano del estado inicial con la perturbacion
#cmts_at_t0 = [1] + [np.random.rand() for i in range(49)]

### Este mÃ³dulo construye la matrix que relaciona los coeficientes c_m(t). Es re contra fea pero it gets the job done

def construct_M_matrix(coeff_list):
    m_matrix_list = []
    ith_cmt_list = []
    M = total_no_spins
    
    ### La ec de markov la podemos escribir como d/dt(C(t)) = -i/I_M^0(t) * M (C(t))
    
    ### la macana es que, con condiciones de borde libres, tengo que cargar a mano los dos primeros y dos Ãºltimos cm(t)s
    ### del lattice. Capaz con un condiciones periÃ³dicas, pensÃ¡ndo la lattice como un Toro T_1, queda mÃ¡s lindo.
    
    m_matrix_list.append(coeff_list[0+2]-coeff_list[0] * np.e**(-0) - (coeff_list[0]) * np.e**(-0) + p * np.e**(-0))
    m_matrix_list.append((coeff_list[1+2]-coeff_list[1]) * np.e**(-1) - (coeff_list[1]) * np.e**(-1)
                       + p * np.e**(-1))
    
    for i in range(2, len(coeff_list) - 2):
        a = (coeff_list[i+2]-coeff_list[i]) * np.e**(-i) + (coeff_list[i-2]-coeff_list[i]) * np.e**(-i) + p * np.e**(-i)
        m_matrix_list.append(a) 
        
    
    m_matrix_list.append((coeff_list[M-2+2-1]-coeff_list[M-2-1]) * np.e**(-(M-2-1)) 
                            - (coeff_list[M-2-2-1]-coeff_list[M-2-1]) * np.e**(-(M-2-1))
                       + p * np.e**(-(M-2-1)))
    
    m_matrix_list.append(-(coeff_list[M-1]) * np.e**(-(M-1)) 
                            - (coeff_list[M-1-2]-coeff_list[M-1]) * np.e**(-(M-1))
                       + p * np.e**(-(M-1)))
    
    m_matrix = np.array(m_matrix_list)
    
    return m_matrix#, m_matrix_list

Mij_matrix = construct_M_matrix(coeff_list_t0)

In [20]:
Mij_matrix

array([-7.16674346e-01, -5.90169624e-01,  1.21698489e-01,  4.81026125e-02,
        6.31353203e-03, -2.51689532e-03,  1.85198030e-04,  1.21991991e-03,
        5.06343237e-04,  6.14551116e-05, -7.23453325e-06,  5.63735127e-06,
       -1.16495815e-06,  7.22258777e-07,  8.17044402e-07,  1.90310080e-07,
        9.73740334e-08, -5.38454793e-09,  1.17394656e-08,  5.36103287e-09,
       -4.30804432e-10,  8.75511904e-10,  2.82353277e-11, -3.94635884e-11,
        4.84266306e-11,  2.40592032e-11,  7.42731834e-13, -2.31364049e-12,
        8.47879926e-13,  5.33238451e-13,  3.29473415e-15, -2.81244765e-14,
        4.42386492e-15,  6.60946827e-15,  1.16701582e-15, -1.08655614e-16,
       -8.27975324e-17,  7.69233729e-17,  5.81487599e-17,  6.35176095e-18,
       -3.87907203e-19,  7.40536971e-19, -1.03335779e-19,  5.66137864e-20,
        1.13789691e-19,  3.79809348e-20, -4.15140553e-22,  5.85889694e-21,
       -1.15054418e-21])

In [46]:
from scipy.integrate import odeint

def F(y, t, H_tensor, coherences_t0_pert0):
    return H_tensor.dot(y)/(coherences_t0_pert0) # * (-1j) primero busco de resolver una ec real que es mÃ¡s facil xd

ts=np.linspace(0,1,200)
result = odeint(func=F, y0=coeff_list_t0,t=ts,args=(Mij_matrix, coherences_t0_pert0, ))
plt.figure(figsize=(4, 3))

for i in [0, 5, 10, 15, 49]:
    plt.plot(result[i], label=r'$\phi_{(%d)}(t)$'%i)

plt.plot(result)
plt.xlabel("t")
plt.ylabel("Phi(t)")
plt.legend()

ValueError: shapes (50,1) and (50,) not aligned: 1 (dim 1) != 50 (dim 0)